In [ ]:
import os
from datetime import datetime
import pandas as pd
from maize_data_scrapper import scrape_data
from tensorflow.keras.models import load_model
import joblib
from functions import (
    load_and_combine_data,
    clean_columns,
    replace_missing_values,
    convert_price_columns,
    impute_missing_values,
    filter_markets,
    remove_outliers,
    export_data,
    feature_engineering_pipeline,
    train_lstm_model)
from predict import predict_and_save_all

NameError: name 'encode_categorical_features' is not defined

In [ ]:
 data = load_and_combine_data(file_paths, scraped_csv_path=scraped_csv_path)
    data = clean_columns(data)
    data = replace_missing_values(data)
    data = convert_price_columns(data, price_columns)
    data = impute_missing_values(data, knn_columns)
    data = data.dropna()
    data.sort_values(by=['County', 'Market', 'Classification', 'Date'], inplace=True)
    data = filter_markets(data, threshold=10)
    data = remove_outliers(data, num_columns)
    data = data.drop_duplicates()

    # Save cleaned data
    clean_data_path = "clean_data2.csv"
    export_data(data, file_name=clean_data_path)

In [ ]:
def automate_pipeline(
    scraped_csv_path="maize_data.csv",
    file_paths=[ 
        "raw data/Market Prices.xls", "raw data/Market Prices 2.xls",
        "raw data/Market Prices 3.xls", "raw data/Market Prices 4.xls",
        "raw data/Market Prices 5.xls", "raw data/Market Prices 6.xls",
        "raw data/Market Prices 7.xls", "raw data/Market Prices 8.xls"
    ],
    price_columns=["Wholesale", "Retail"],
    knn_columns=["Supply Volume", "Retail", "Wholesale"],
    num_columns=["Retail", "Wholesale", "Supply Volume"],
    forecast_horizon=20,
    counties=['Baringo', 'Bomet', 'Bungoma', 'Busia', 'Elgeyo-Marakwet', 'Embu',
       'Garissa', 'Homa-bay', 'Isiolo', 'Kajiado', 'Kakamega', 'Kericho',
       'Kiambu', 'Kilifi', 'Kirinyaga', 'Kisii', 'Kisumu', 'Kitui',
       'Kwale', 'Laikipia', 'Lamu', 'Machakos', 'Makueni', 'Mandera',
       'Meru', 'Migori', 'Mombasa', 'Muranga', 'Nairobi', 'Nakuru',
       'Nandi', 'Narok', 'Nyamira', 'Nyandarua', 'Nyeri', 'Samburu',
       'Siaya', 'Taita-Taveta', 'Tana-River', 'Tharaka-Nithi',
       'Trans-Nzoia', 'Turkana', 'Uasin-Gishu', 'Vihiga', 'Wajir',
       'West-Pokot'],
    prediction_save_path="predictions.csv"
):
    # Step 1: Scrape data
    print("Step 1: Scraping data...")
    scrape_data()

    # Step 2: Combine and clean data
    print("Step 2: Cleaning data...")
    data = load_and_combine_data(file_paths, scraped_csv_path=scraped_csv_path)
    data = clean_columns(data)
    data = replace_missing_values(data)
    data = convert_price_columns(data, price_columns)
    data = impute_missing_values(data, knn_columns)
    data = data.dropna()
    data.sort_values(by=['County', 'Market', 'Classification', 'Date'], inplace=True)
    data = filter_markets(data, threshold=10)
    data = remove_outliers(data, num_columns)
    data = data.drop_duplicates()

    # Save cleaned data
    clean_data_path = "clean_data2.csv"
    export_data(data, file_name=clean_data_path)

    # Step 3: Perform feature engineering
    print("Step 3: Feature engineering...")
    feature_engineering_pipeline()

    # Step 4: Retrain models
    print("Step 4: Retraining models...")

    # Wholesale model
    train_lstm_model(
        target_variable='Wholesale',
        scaler_feature_path="models/scaler_wholesale_features.pkl",
        scaler_target_path="models/scaler_wholesale_target.pkl",
        model_save_path="models/best_wholesale_model_sequence.h5"
    )
    # Retail model
    train_lstm_model(
        target_variable='Retail',
        scaler_feature_path="models/scaler_retail_features.pkl",
        scaler_target_path="models/scaler_retail_target.pkl",
        model_save_path="models/best_retail_model_sequence.h5"
    )

    # Step 5: Generate predictions using the new predict_and_save function
    print("Step 5: Generating predictions...")

    predict_and_save_all(
    current_date="2024-12-11",
    reference_data=pd.read_csv("historical_data.csv"),
    wholesale_model=tf.keras.models.load_model(r"models/best_wholesale_model_sequence.h5"),
    retail_model=tf.keras.models.load_model(r"models/best_retail_model_sequence.h5"),
    wholesale_scalers=(joblib.load("models/scaler_wholesale_features.pkl"), joblib.load("models/scaler_wholesale_target.pkl")),
    retail_scalers=(joblib.load("models/scaler_retail_features.pkl"), joblib.load("models/scaler_retail_target.pkl")),
    encoder=joblib.load("models/County_binary_encoder.pkl"),
    counties=[
    'Baringo', 'Bomet', 'Bungoma', 'Busia', 'Elgeyo-Marakwet', 'Embu',
    'Garissa', 'Homa-bay', 'Isiolo', 'Kajiado', 'Kakamega', 'Kericho',
    'Kiambu', 'Kilifi', 'Kirinyaga', 'Kisii', 'Kisumu', 'Kitui',
    'Kwale', 'Laikipia', 'Lamu', 'Machakos', 'Makueni', 'Mandera',
    'Meru', 'Migori', 'Mombasa', 'Muranga', 'Nairobi', 'Nakuru',
    'Nandi', 'Narok', 'Nyamira', 'Nyandarua', 'Nyeri', 'Samburu',
    'Siaya', 'Taita-Taveta', 'Tana-River', 'Tharaka-Nithi',
    'Trans-Nzoia', 'Turkana', 'Uasin-Gishu', 'Vihiga', 'Wajir',
    'West-Pokot'
    ],
    output_file="predictions.csv")

    print("Automation complete! Predictions saved for all counties and models retrained.")